In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [3]:
## drop the columns
df = df.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)

In [4]:
## encode the categorical variable
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])

In [5]:
## One-hot encoder
#create encoder
onehotencoder = OneHotEncoder(sparse_output=False)

# fit and transform
encoded = onehotencoder.fit_transform(df[['Geography']])

## get feature names
feature_names = onehotencoder.get_feature_names_out(['Geography'])

## create data frame with encoded columns
encoded_df = pd.DataFrame(encoded, columns=feature_names, index=df.index)

## drop original column and concatenate encoded columns
df = df.drop('Geography',axis=1)
df = pd.concat([df, encoded_df], axis=1)

df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [6]:
## independent features
X = df.drop('EstimatedSalary', axis=1)
## dependent feature
y = df['EstimatedSalary']

In [7]:
## train test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
## scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
## save the pickle files
with open('encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

with open('onehotencoder.pkl','wb') as file:
    pickle.dump(onehotencoder,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

In [10]:
## train ANN

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
]
)

## compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

d:\End-to-end-Projects\6-DL-project\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

## setup the tensorboard
log_dir = 'regressionlogs/fit/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq = 1)

In [13]:
## setup warly stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [14]:
## train the model
history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs=100,
    callbacks=[early_stopping, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 65ms/step - loss: 100371.3125 - mae: 100371.3125 - val_loss: 98489.9531 - val_mae: 98489.9531
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 42ms/step - loss: 99529.7969 - mae: 99529.7969 - val_loss: 96787.6250 - val_mae: 96787.6250
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - loss: 96529.2891 - mae: 96529.2891 - val_loss: 92335.8594 - val_mae: 92335.8594
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - loss: 90578.7422 - mae: 90578.7422 - val_loss: 84951.2109 - val_mae: 84951.2109
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 82028.3516 - mae: 82028.3516 - val_loss: 75725.5312 - val_mae: 75725.5312
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - loss: 72314.0078 - mae: 72314.0078 - val_loss: 66457.3906 - val_mae: 66457.3906
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 63357.3477 - mae: 63357.3477 - val_loss: 59039.2773 - val_mae: 59039.2773
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 

In [15]:
%load_ext tensorboard

In [16]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 14424), started 0:39:16 ago. (Use '!kill 14424' to kill it.)

In [17]:
## save the model

model.save('model_reg.keras')